### 1、RPC

* 概念：抽象的概念，远程过程调用；http通讯也是远程调用；
  * RPC应用：对内，不同计算机通讯，一个项目中，不同的功能模块之间的调用，比如：Flask调用推荐系统，Django调用聊天机器人；
  * HTTP应用：对外，不同计算机通讯，适用性更强；
    * HTTP更多的面向用户与产品服务器的通讯。
    * RPC更多的面向产品内部服务器间的通讯。
  * 广义上：所有通过网络来进行通讯调用的实现统称为RPC；
  * 狭义上：区别于HTTP的实现方式，在传输的**数据格式**上和**传输的控制**上独立实现；
* 特点：
  * 效率高
  * 发起RPC调用的一方，在编写代码时可忽略RPC的具体实现，如同编写本地函数调用一样
  * 缺点：通用性不如HTTP好，因为传输的数据不是HTTP协议格式，所以调用双方需要专门实现的通信库，对于不同的编程开发语言，都要有相关实现。

### 2、gRPC

* 概念：实现rpc的一个框架，Google公司开源的高性能RPC框架。grpc和rpc的关系，相当于django和http的关系。
* 特点：
  * 1、多语言
  * 2、多平台
  * 3、**Protocol Buffers协议机制（proto3） 序列化**，使用grpc需要定义proto格式的文件。
  * 4、支持双向流和连接多路复用
    * 单工：发送方，接收方，只能一方发，一方收；
    * 半双工：发送方，接收方，可以一方发，另一方收，不能同时进行；
    * 全双工：发送方，接收方，可以一方发，另一方收，可以同时进行；



* 安装使用：

~~~python
# 用来构建rpc服务器和客户端
pip install grpc
# 用来把proto文件转成py文件
pip install grpcio-tools
~~~



* ##### Protocol Buffers

  * 概念：协议机制，类似于一门语法，标识如何定义功能和数据；
  * 基本语法：在proto后缀名的文件中使用；
    * 1、第一行可以声明版本：**syntax="proto3";**
    * 2、定义数据：**关键字message**，相当于python中定义字典、列表、变量、元组等；
    * 3、定义功能：**关键字service**，rpc定义服务，相当于python中定义类、函数；
    * 4、重复数据：**关键字repeated**表示定义重复数据，相当于python中的列表；



### 3、推荐系统接口分析

* 需求：首页文章列表接口
* 如何设计？

~~~python
# 发送请求
请求参数 = {
  user_id:xxx, # 登录用户或匿名用户
  channel_id:xxx, # 用户选择的频道
  article_num:xxx, # 文章数量
  time_stamp:xxx # 时间戳
}

# 返回响应
响应数据 = {
  exposure=[‘点击文章，分享文章...’] # 曝光参数，记录用户操作的信息
  time_stamp:xxx,
  article:[
    {
      article_id:1,
      content:xx,
      track:{ # 表示埋点数据
        click,
        share,
        collect,
        read,
        liking
        ...
      }
    },
    {
      article_id:2,
      content:xx,
      track:{ # 表示埋点数据
        click,
        share,
        collect,
        read,
        liking
        ...
      }
    },
  ]
}


~~~



* grpc实现步骤：

  * 1、根据接口原型，定义proto文件，定义请求信息、服务方法；

  ~~~python
  # 可以不声明，默认是proto2
  syntax="proto3";
  # message表示数据，相当于python中定义字典、列表、元组、变量等
  message UserRequest{
      string user_id=1;
      int32 channel_id=2;
      int32 article_num=3;
      int64 time_stamp=4;
  }
  
  message ArticleResponse{
      string exposure=1;
      int64 time_stamp=2;
      repeated Article recommends=3;
  }
  
  message Article{
      int32 article_id=1;
      Track track=2;
  }
  
  message Track{
      string click=1;
      string collect=2;
      string share=3;
      string read=4;
  }
  
  # 表示定义服务，相当于python中定义类和函数
  service UserRecommends{
      rpc user_recommend(UserRequest) returns(ArticleResponse){}
  }
  
  ~~~

  * 2、把reco.proto文件转成python文件，在rpc文件夹下执行指令

  ```shell
  # 必须在centOS系统的虚拟环境执行，workon toutiao
  python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. reco.proto
  # 指令执行后，如果没有异常，会生成2个文件
  reco_pb2.py # 保存了数据，message定义的内容；
  reco_pb2_grpc.py # 保存了服务，service定义的内容；
  ```

  * 3、在rpc文件夹下，鼠标右键deployment下载代码文件；



### 4.即时通讯

* 概念：基于互联网的即时交流消息的业务；
* 实现方式：
  * 1、在线push
    * 适用：web页面 和 App
  * 2、离线push
    * 适用：App

* 应用场景：
  * 用户下了订单，需要在运营管理后台向运营人员推送新订单通知
  * 用户A关注了用户B，系统需要向用户B推送提示消息
  * 即时聊天

* **websocket协议**：**HTML5定义了WebSocket协议**，能更好的节省服务器资源和带宽，并且能够更实时地进行通讯。
  * WebSocket是一种在单个TCP连接上进行全双工通信的协议
  * **浏览器和服务器只需要完成一次握手（不是指建立TCP连接的那个三次握手，是指在建立TCP连接后传输一次握手数据），两者之间就直接可以创建持久性的连接，并进行双向数据传输。**

* rpc是实现不同计算机之间的数据传输，先有请求再有响应，传输数据效率很高；
* 即时通讯实现服务器主动发送数据给客户端；

### 5、socket.IO

* 概念：**Socket.IO 本是一个面向实时 web 应用的 JavaScript 库，现在已成为拥有众多语言支持的Web即时通讯应用的框架。**
  * **Socket.IO 不等价于 WebSocket**
  * Socket.io要求客户端与服务器端均须使用该框架



* 创建即时通讯服务器：

  * 1、多线程、多进程方式

  * 2、作为web应用的一部分

  * 3、协程方式-----推荐方式，

    * 协程：微线程；
    * eventlet：异步的协程库

    ~~~python
    def fun_a():
      print("a")
      for x in range(1000):
        print("b")
        if x == 500:
    			yield fun_b # 表示代码执行到yield暂停，yield会保存执行状态；
          print('c')
    
    f1 = fun_a()
    next(f1) # 继续执行代码
    
    # gevent封装了yield
    import eventlet
    eventlet.monkey_patch() # 猴子补丁，作用：补丁下面的代码如果有阻塞会自动切换。
    
    f = open('xx.mp4','rb')
    data = f.read() # python自带的read函数，如果有补丁，read已经被gevent模块提供的同名函数替换掉
    request.post(f.read())
    ~~~

    

* 编写事件：
  * 默认事件：connect、disconnect、message；











